In [ ]:
colab = False

if colab:
    from google.colab import drive
    drive.mount('/content/gdrive')
    !git clone https://github.com/deepmind/pycolab.git
    !git clone https://github.com/nicoladainese96/RelationalModule.git
    !pip install pycolab

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from RelationalModule import ActorCritic, ControlActorCritic
from RelationalModule import train_agent as train
from RelationalModule import utils
from importlib import reload

In [ ]:
reload(train)

In [ ]:
GRID_SIZE = 7
game_params = dict(grid_size=GRID_SIZE,
                solution_length=[0],
                num_forward = [0], # number of distractors
                num_backward=[0], # just set to 0 for now
                branch_length=1, # length of forward distractors
                max_num_steps = 50
               )

In [ ]:
HPs = dict(action_space=4, lr=0.003, gamma=0.99, TD=True, twin=True, tau=0.2, n_steps=40,
           n_kernels=96, vocab_size = 117, n_dim=12, n_features=64, n_heads=4, n_attn_modules=2, 
           n_linears=4, max_pool=False, linear_size=GRID_SIZE+2)
if colab:
    HPs['device'] = 'cuda'
else:
    HPs['device'] = 'cpu'

print('device: ', HPs['device'])  

# Relational Agent
agent = ActorCritic.BoxWorldA2C(**HPs)

control_HPs = dict(action_space=4, lr=0.003, gamma=0.99, TD=True, twin=True, tau=0.2, n_steps=40)
           
# Control Agent
control_agent = ControlActorCritic.ControlA2C(**control_HPs)

In [ ]:
# Random Agent

class RandomAgent():
    def __init__(self, n_actions):
        self.n_actions = n_actions
    
    def get_action(self,state, *args, **kwargs):
        a = np.random.choice(self.n_actions)
        log_prob = np.log(1./self.n_actions) # just because it's the standard output of the other agent
        return a, log_prob
    
    def update(self, *args):
        return

rnd_agent = RandomAgent(4)

In [ ]:
%%time
results = train.train_boxworld(agent, game_params, n_episodes=5000, 
                               max_steps=game_params['max_num_steps'], return_agent=True)
score, asymptotic_score, asymptotic_std, trained_agent, time_profile = results

In [ ]:
plt.figure(figsize=(8,6))
n_epochs = np.arange(100, len(score))
average_score = np.array([np.mean(score[i:i+100]) for i in range(len(score)-100)])
plt.plot(n_epochs, average_score, alpha=0.9)
plt.title("Performance", fontsize=16)
plt.xlabel("Number of epochs", fontsize=16)
plt.ylabel("Total reward", fontsize=16)
plt.show()

In [ ]:
save = False
keywords = ['relational', 'residual','unboxed_gem',str(len(control_score))+"-episodes","50-steps"] # example

if colab and save:
    %cd ~
    parent_dir = "/content/gdrive/My Drive/Colab Notebooks/"
    save_dir  = "RelationalTrained/"
    %cd "{parent_dir}"
    !mkdir "{save_dir}"
    ID = utils.save_session(save_dir, keywords, game_params, HPs, score)
    torch.save(trained_agent, save_dir+"agent_"+ID)